In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("white")

# Carga y limpieza de datos

In [ ]:
raw_data = pd.read_csv('./data/ventas.csv', encoding='utf-8', sep=',')

In [ ]:
raw_data.info()

In [ ]:
# Corrigiendo formato de fecha
raw_data['fecha'] = pd.to_datetime(raw_data['fecha'], format='%Y-%m-%d')

In [ ]:
raw_data.describe()

In [ ]:
raw_data.value_counts('producto')

In [ ]:
for product in raw_data['producto'].unique():
    print("Métricas para el producto:", product)
    print(raw_data[raw_data['producto'] == product].describe())
    print("\n")

In [ ]:
# Se verifica que no exista productos duplicados en una misma fecha, no tendría sentido que exista más de una vez el mismo producto en una misma fecha a menos que el precio unitario sea diferente.
raw_data[raw_data.duplicated(subset=['fecha'], keep=False)].sort_values(by='fecha')

Se verifica que la data es consistente y no presenta valores nulos o fuera de lo común.
De igual forma se aplican los filtro necesarios para evitar que en data futura se presenten errores.

In [ ]:
data_cleaned = raw_data[(raw_data['cantidad'] > 0) & (raw_data['precio_unitario'] > 0)]
data_cleaned.dropna(inplace=True)

In [ ]:
# Creando la columna 'total' para el total de ventas de cada producto para su respectiva fecha
data_cleaned['total'] = data_cleaned['cantidad'] * data_cleaned['precio_unitario']

In [ ]:
data_cleaned.sort_values(by='fecha', inplace=True)

In [ ]:
data_cleaned.info()

In [ ]:
data_cleaned.to_csv('data/data_cleaned.csv', index=False)

# Análisis

**(a) Producto más vendido (en cantidad de unidades):**

In [ ]:
data_cleaned.groupby('producto')['cantidad'].sum().sort_values(ascending=False)

**(b) Producto más vendido (en valor total):**

In [ ]:
data_cleaned.groupby('producto')['total'].sum().sort_values(ascending=False)

**(c) Facturación total por mes:**

In [ ]:
data_cleaned.groupby(data_cleaned['fecha'].dt.to_period('M'))['total'].sum()

# Visualizaciones

In [ ]:
facturacion_por_producto = data_cleaned.groupby('producto')['total'].sum().sort_values(ascending=False)
sns.barplot(data=facturacion_por_producto.reset_index(), x='producto', y='total');

In [ ]:
facturacion_mensual = data_cleaned.groupby(data_cleaned['fecha'].dt.to_period('M'))['total'].sum()
sns.barplot(data=facturacion_mensual.reset_index(), x='fecha', y='total')
plt.title("Facturación total por mes")
plt.xlabel("Mes")
plt.ylabel("Facturación Total")
plt.savefig('output/grafico_mvp.png')

In [ ]:
facturacion_mensual_producto = data_cleaned.groupby([data_cleaned['fecha'].dt.to_period('M'), 'producto'])['total'].sum()
facturacion_mensual_producto = facturacion_mensual_producto.reset_index()
facturacion_mensual_producto['fecha'] = facturacion_mensual_producto['fecha'].dt.to_timestamp()
facturacion_mensual_producto

In [ ]:
sns.lineplot(data=facturacion_mensual_producto, x='fecha', y='total', hue='producto');

# Preparación DB

In [ ]:
reporte_productos = data_cleaned.groupby('producto').agg(total_cantidad=('cantidad', 'sum'), total_facturado=('total', 'sum'))
reporte_productos

In [ ]:
import sqlite3
try:
    # Creando conexión a la base de datos SQLite
    conn = sqlite3.connect('output/db_test.db')
    # Guardando el dataframe en la base de datos
    reporte_productos.to_sql('reporte_productos', conn, if_exists='replace', index=False)
    print("Prueba exitosa: La tabla 'reporte_productos' se guardó exitosamente.")

    # Verificando que los datos se guardaron correctamente
    test_db = pd.read_sql('SELECT * FROM reporte_productos', conn)
    print(test_db.head())
    conn.close()

except Exception as e:
    print(f"Ocurrió un error durante la prueba de base de datos: {e}")